In [1]:
# lectura de datos 
from ssconnect import connect_singlestore
import pandas as pd
import datetime as dt


engine = connect_singlestore()
sql="SELECT * FROM DWH.VW_VENTA_VS_MANTENIMIENTO WHERE FAMILIA_VH NOT IN ('NO DEFINIDO')"
df_prep= pd.read_sql(sql, engine)
df_prep = df_prep[df_prep['TIPO_DOCUMENTO'] == 'FACTURA CREDITO']
df_prep = df_prep.sort_values('FECHA', ascending=False)
df_prep = df_prep.drop_duplicates(subset=['CHASIS'], keep='first')
df_prep['FECHA'] = pd.to_datetime(df_prep['FECHA'], format='%Y-%m-%d', errors='coerce')
df_prep = df_prep[df_prep['FECHA'] < dt.datetime(2025, 1, 1)]
df_prep.reset_index(drop=True, inplace=True)

# Convertir las columnas de fecha a tipo datetime
df_prep['FECHA_SALIDA'] = pd.to_datetime(df_prep['FECHA_SALIDA'], format='%Y-%m-%d', errors='coerce')
df_prep['FECHA_MNT_1000'] = pd.to_datetime(df_prep['FECHA_MNT_1000'], format='%Y-%m-%d', errors='coerce')
df_prep['FECHA_MNT_1000_2'] = pd.to_datetime(df_prep['FECHA_MNT_1000'], format='%Y-%m-%d', errors='coerce')
# Si FECHA_MNT_1000 es 1970-01-01 , colocar la fecha de hoy
df_prep['FECHA_MNT_1000_2'] = df_prep['FECHA_MNT_1000_2'].where(df_prep['FECHA_MNT_1000_2'] > dt.datetime(1970, 1, 1), dt.datetime.today())
# Calcular la diferencia en días entre LA FECHA  y el primer mantenimiento
df_prep['TIEMPO_MANT_1000'] = (df_prep['FECHA_MNT_1000_2'] - df_prep['FECHA']).dt.days
# Crear una columna de evento (si hubo mantenimiento o no)
df_prep['EVENTO_MANT_1000'] = df_prep['MANTENIMIENTO_1000'].apply(lambda x: 1 if x > 0 else 0)
# quitar los registos con timepo mant negativos
df_prep = df_prep[df_prep['TIEMPO_MANT_1000'] >= 0]
# los registros que tienen mas de 120 dias hasta el primer mantenimiento se consideran censurados
df_prep['EVENTO_MANT_1000'] = df_prep['EVENTO_MANT_1000'].where(df_prep['TIEMPO_MANT_1000'] <= 120, 0)

#crear la columna de censura, si el tiempo hasta el primer mantenimiento es mayor a 120 dias se considera censurado de igual manera los que tienen fecha de mantenimiento 1970-01-01
df_prep['CENSURA'] = df_prep['TIEMPO_MANT_1000'].apply(lambda x: 1 if x > 120 else 0)
# los registros que tienen fecha de mantenimiento 1970-01-01 se consideran censurados   
df_prep['CENSURA'] = df_prep['CENSURA'].where(df_prep['FECHA_MNT_1000'] > dt.datetime(1970, 1, 1), 1)
# los registros que son iguales o mayores a 120 dias se considera que no tuvieron mantenimiento
df_prep['TIEMPO_MANT_1000'] = df_prep['TIEMPO_MANT_1000'].where(df_prep['TIEMPO_MANT_1000'] <= 120, 120)
# seleccionar las columnas necesarias para el analisis 
df_prep = df_prep[['CHASIS','TIEMPO_MANT_1000', 'EVENTO_MANT_1000','CENSURA', 'AGENCIA', 'LINEA_NEGOCIO', 'FAMILIA_VH']]
df_prep.head()


,CHASIS,TIEMPO_MANT_1000,EVENTO_MANT_1000,CENSURA,AGENCIA,LINEA_NEGOCIO,FAMILIA_VH
0,9BRKZAAG4S0701975,94,1,0,GRANADOS,VEH. NUEVOS TOYOTA,COROLLA CROSS
1,JTDKBABB4SA427735,80,1,0,LOS CHILLOS,VEH. NUEVOS TOYOTA,YARIS CROSS
2,JTDKBABB5SA417389,83,1,0,SUR,VEH. NUEVOS TOYOTA,YARIS CROSS
3,MHFBU3FS0S0352770,101,0,1,SANTO DOMINGO,VEH. NUEVOS TOYOTA,FORTUNER
4,8AJCB3DD2S3920542,101,0,1,GRANADOS,VEH. NUEVOS TOYOTA,HILUX


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sklearn.preprocessing import OneHotEncoder
import numpy as np


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [10:19:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Model Evaluation:
              precision    recall  f1-score   support

           0       0.50      0.01      0.01      1356
           1       0.69      1.00      0.82      3065

    accuracy                           0.69      4421
   macro avg       0.60      0.50      0.41      4421
weighted avg       0.63      0.69      0.57      4421

Random Forest Model Evaluation:
              precision    recall  f1-score   support

           0       0.50      0.01      0.01      1356
           1       0.69      1.00      0.82      3065

    accuracy                           0.69      4421
   macro avg       0.60      0.50      0.41      4421
weighted avg       0.63      0.69      0.57      4421

Gradient Boosting Model Evaluation:
              precision    recall  f1-score   support

           0       0.50      0.00      0.01      1356
           1       0.69      1.00      0.82      3065

    accuracy                           0.69      4421
   macro avg       0.60      0.50 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVC Model Evaluation:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1356
           1       0.69      1.00      0.82      3065

    accuracy                           0.69      4421
   macro avg       0.35      0.50      0.41      4421
weighted avg       0.48      0.69      0.57      4421



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa

Voting Classifier Model Evaluation:
              precision    recall  f1-score   support

           0       0.50      0.00      0.01      1356
           1       0.69      1.00      0.82      3065

    accuracy                           0.69      4421
   macro avg       0.60      0.50      0.41      4421
weighted avg       0.63      0.69      0.57      4421

XGBoost Accuracy: 0.6933
Random Forest Accuracy: 0.6933
Gradient Boosting Accuracy: 0.6933
Logistic Regression Accuracy: 0.6933
SVC Accuracy: 0.6933
Voting Classifier Accuracy: 0.6933


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Asumiendo que df_prep ya está cargado
# df_prep = ...

# Seleccionar las variables relevantes
df = df_prep[['TIEMPO_MANT_1000', 'EVENTO_MANT_1000', 'AGENCIA', 'FAMILIA_VH']].copy()
df.dropna(inplace=True) # Eliminar filas con valores faltantes para simplificar

# Codificación one-hot para variables categóricas
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_cols = ['AGENCIA', 'FAMILIA_VH']
encoded_data = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

# Resetear el índice de df para que coincida con encoded_df después de la transformación
df_reset = df.reset_index(drop=True)
df_processed = pd.concat([df_reset[['TIEMPO_MANT_1000', 'EVENTO_MANT_1000']], encoded_df], axis=1)

# Crear el array estructurado de supervivencia
y = np.array(list(zip(df_processed['EVENTO_MANT_1000'], df_processed['TIEMPO_MANT_1000'])),
             dtype=[('status', '?'), ('time', '<f8')])

# Separar características (X) y variable de supervivencia (y)
X = df_processed.drop(columns=['TIEMPO_MANT_1000', 'EVENTO_MANT_1000'])

# Dividir los datos en conjuntos de entrenamiento y prueba , aplicar sobremuestreo 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
from imblearn.over_sampling import SMOTE
# Aplicar SMOTE para el sobremuestreo
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
# Convertir y_train_resampled a un array estructurado
y_train_resampled = np.array(list(zip(y_train_resampled['status'], y_train_resampled['time'])),
                             dtype=[('status', '?'), ('time', '<f8')])
# Verificar la forma de los datos
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_train_resampled shape: {X_train_resampled.shape}, y_train_resampled shape: {y_train_resampled.shape}")


# Inicializar y entrenar el modelo GradientBoostingSurvivalAnalysis , crear una for que iteractue entre diferentes parametos
estimadors=[50, 100, 200,]
lr=[0.01,  0.1]
c_indexes = []

for estimador in estimadors:
    for learning_rate in lr:
        print(f"Estimadores: {estimador}, Learning Rate: {learning_rate}")
        gbsa = GradientBoostingSurvivalAnalysis(n_estimators=estimador, learning_rate=learning_rate, random_state=42)
        gbsa.fit(X_train, y_train)
        # Predecir los riesgos en el conjunto de prueba
        survival_risks = gbsa.predict(X_test)
        # Calcular el índice de concordancia en el conjunto de prueba , agregar los parametros del modelo
        c_index = concordance_index_censored(y_test['status'], survival_risks, y_test['time'])
        c_indexes.append(c_index[0])
        print(f"Estimadores: {estimador}, Learning Rate: {learning_rate}, C-index: {c_index[0]:.4f}")









X_train shape: (16577, 25), y_train shape: (16577,)
X_train_resampled shape: (23056, 25), y_train_resampled shape: (16577,)
Estimadores: 50, Learning Rate: 0.01


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


ValueError: Found input variables with inconsistent numbers of samples: [23056, 16577, 16577]

In [18]:
# Crear un DataFrame para almacenar los resultados
results_df = pd.DataFrame({
    'Estimadores': [f"{estimator}" for estimator in estimadors for _ in range(len(lr))],
    'Learning Rate': [f"{learning_rate}" for _ in estimadors for learning_rate in lr],
    'C-index': c_indexes
})
results_df = results_df.sort_values(by='C-index', ascending=False)
# SELECCIONAR EL MEJOR MODELO
best_model_index = results_df['C-index'].idxmax()
best_model = results_df.iloc[best_model_index]
best_estimators = best_model['Estimadores']
best_learning_rate = best_model['Learning Rate']
best_c_index = best_model['C-index']
print(f"Mejor modelo: Estimadores: {best_estimators}, Learning Rate: {best_learning_rate}, C-index: {best_c_index:.4f}")

 


Mejor modelo: Estimadores: 50, Learning Rate: 0.01, C-index: 0.3349


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# graficar curva de supervivencia total
def plot_survival_curve(model, X, y, title="Curva de Supervivencia"):
    # Predecir la función de supervivencia para cada individuo
    survival_function = model.predict_survival_function(X)
    
    # Graficar la curva de supervivencia
    plt.figure(figsize=(10, 6))
    for i in range(len(survival_function)):
        plt.step(survival_function[i].x, survival_function[i].y, where="post", label=f"Individuo {i+1}")
    
    plt.title(title)
    plt.xlabel("Tiempo")
    plt.ylabel("Probabilidad de Supervivencia")
    plt.legend()
    plt.grid()
    plt.show()
# Graficar la curva de supervivencia total
plot_survival_curve(gbsa, X_test, y_test, title="Curva de Supervivencia Total")
# Graficar la curva de supervivencia en funcion de la familia de vh
def plot_survival_curve_by_family(model, X, y, family_col, title="Curva de Supervivencia por Familia"):
    # Predecir la función de supervivencia para cada individuo
    survival_function = model.predict_survival_function(X)
    
    # Graficar la curva de supervivencia por familia
    plt.figure(figsize=(10, 6))
    unique_families = X[family_col].unique()
    
    for family in unique_families:
        family_mask = X[family_col] == family
        for i in np.where(family_mask)[0]:
            plt.step(survival_function[i].x, survival_function[i].y, where="post", label=f"{family} - Individuo {i+1}")
    
    plt.title(title)
    plt.xlabel("Tiempo")
    plt.ylabel("Probabilidad de Supervivencia")
    plt.legend()
    plt.grid()
    plt.show()
# Graficar la curva de supervivencia por agencia
plot_survival_curve_by_family(gbsa, X_test, y_test, 'FAMILIA_VH', title="Curva de Supervivencia por Familia")

